In [6]:
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from dotenv import load_dotenv

In [7]:
load_dotenv()

True

In [8]:
memory = MemorySaver()
model = ChatAnthropic(model_name="claude-3-5-sonnet-20241022")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [9]:
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

hi im bob! and i live in sf
================================== Ai Message ==================================

Hello Bob from San Francisco! I don't see a specific question or request from you that requires using the available tools. I have access to a search tool that can help answer questions about current events and provide comprehensive information. 

Is there something specific you'd like to know about San Francisco or any other topic? I'd be happy to help search for relevant information for you!


In [10]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather where I live?
================================== Ai Message ==================================

[{'text': 'Let me search for current weather information in San Francisco.', 'type': 'text'}, {'id': 'toolu_013MenUZz4n5RVuQ3iGYPJMG', 'input': {'query': 'current weather San Francisco CA'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_013MenUZz4n5RVuQ3iGYPJMG)
 Call ID: toolu_013MenUZz4n5RVuQ3iGYPJMG
  Args:
    query: current weather San Francisco CA
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Weather in San Francisco, CA", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_e

In [11]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [12]:
search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)

[{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1744506213, 'localtime': '2025-04-12 18:03'}, 'current': {'last_updated_epoch': 1744506000, 'last_updated': '2025-04-12 18:00', 'temp_c': 16.7, 'temp_f': 62.1, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 10.5, 'wind_kph': 16.9, 'wind_degree': 268, 'wind_dir': 'W', 'pressure_mb': 1016.0, 'pressure_in': 30.01, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 52, 'cloud': 25, 'feelslike_c': 16.7, 'feelslike_f': 62.1, 'windchill_c': 15.8, 'windchill_f': 60.4, 'heatindex_c': 15.8, 'heatindex_f': 60.4, 'dewpoint_c': 13.4, 'dewpoint_f': 56.2, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.7, 'gust_mph': 14.4, 'gust_kph': 23.1}}", 'score'

In [13]:
tools = [search]

In [14]:
from langchain.chat_models import init_chat_model

In [15]:
model = init_chat_model("gpt-4", model_provider="openai")

In [23]:
from langchain_core.messages import HumanMessage, SystemMessage

In [17]:
response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [18]:
model_with_tools = model.bind_tools(tools)

In [19]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [20]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: I'm sorry for any confusion, but as an AI, I'm not equipped to answer real-time or current event questions like current weather reports. However, you can use weather forecast websites or apps to get the weather updates you need.
ToolCalls: []


In [21]:
tools

[TavilySearchResults(max_results=2, api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********')))]

In [24]:
response = model_with_tools.invoke([
    SystemMessage(content="You have access to a search tool for answering real-time questions."),
    HumanMessage(content="What's the weather in SF?")
])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_BsSueoNs3ZEIKqq7KldTZJ7l', 'type': 'tool_call'}]


In [25]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [27]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi")]})
response["messages"]

[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}, id='04542a87-bbcd-45c3-92b0-1fbae7a7a1f3'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 82, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BLgQBGwB1phWnJytUwZdmp1UzO3PE', 'finish_reason': 'stop', 'logprobs': None}, id='run-e8f50e54-c198-4637-8988-d27903d6036b-0', usage_metadata={'input_tokens': 82, 'output_tokens': 11, 'total_tokens': 93, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In [28]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_PY9qzfEeTaEkTlXnKStpBlXz)
 Call ID: call_PY9qzfEeTaEkTlXnKStpBlXz
  Args:
    query: current weather in San Francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Sunday, April 13, 2025. San Francisco, CA - Weather Forecast", "url": "https://weathershogun.com/weather/usa/ca/san-francisco/480/april/2025-04-13", "content": "San Francisco, California Weather: Sunday, April 13, 2025. Sunny weather, clear skies and sunny weather. Day 61°. Night 46°. Precipitation 0 %.", "score": 0.93898576}, {"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', '

In [29]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The| current| weather| in| San| Francisco|,| California|,| as| of| Sunday|,| April| |13|,| |202|5|,| is| sunny|,| with| clear| skies| and| a| temperature| of| |62|.|1|°F| during| the| day| and| |46| °|F| at| night|.| There|'s| no| precipitation|.| The| wind| is| coming| from| the| West| at| a| speed| of| |11|.|2| mph|.| [|Source|](|https|://|www|.weather|api|.com|/)
|

In [30]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [31]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [32]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("---")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 85, 'total_tokens': 97, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BLgddY9CAunaQjYZW9mmEjAcSPzjG', 'finish_reason': 'stop', 'logprobs': None}, id='run-13456e6c-f70e-4bed-9723-efb1762108dd-0', usage_metadata={'input_tokens': 85, 'output_tokens': 12, 'total_tokens': 97, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
---


In [34]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("---")

{'agent': {'messages': [AIMessage(content='Your name is Bob.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 108, 'total_tokens': 115, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BLgekz5bCg6QJjj7CQPBZU4OLAOaL', 'finish_reason': 'stop', 'logprobs': None}, id='run-731f0656-1dcb-4bd3-a249-61bc20cf4548-0', usage_metadata={'input_tokens': 108, 'output_tokens': 7, 'total_tokens': 115, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
---


In [35]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("---")

{'agent': {'messages': [AIMessage(content="As an AI model developed by OpenAI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality. So, I don't know your name.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 86, 'total_tokens': 140, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BLggDHpBnChVZIeLgWcnTMliO37xx', 'finish_reason': 'stop', 'logprobs': None}, id='run-842b014d-9d6f-4134-a8e1-ee6be9d4488e-0', usage_metadata={'input_tokens': 86, 'output_tokens': 54, 'total_tokens': 140, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 